In [51]:
import pickle
import numpy as np
import pandas as pd
import scipy.io
from tqdm import tqdm
from scipy.stats import permutation_test

In [4]:
nov = scipy.io.loadmat("/home/zachkaras/fmri/midprocessing/nov_connectivity.mat")
int = scipy.io.loadmat("/home/zachkaras/fmri/midprocessing/int_connectivity.mat")
exp = scipy.io.loadmat("/home/zachkaras/fmri/midprocessing/exp_connectivity.mat")

In [21]:
nov_data = nov['nov_connectivity']
int_data = int['int_connectivity']
exp_data = exp['exp_connectivity']

In [52]:
def test_statistic(x,y):
    return np.mean(x) - np.mean(y)

def run_permutation_test(group1, group2):
    All_Seeds_Stats = []
    All_Seeds_Pvals = []
    for x in range(group1.shape[0]): # iterating through seed regions
        stats = [0]*group1.shape[1]
        pvals = [0]*group1.shape[1]
        print(f"seed {x} permutation testing...")
        for y in tqdm(range(group1.shape[1]), desc="Processing voxels"): # voxels
            group1_list = group1[x,y,:] # usually novices
            group2_list = group2[x,y,:] # either intermediates or experts
            result = permutation_test((group1_list, group2_list), test_statistic, alternative='two-sided')
            stats[y] = result.statistic
            pvals[y] = result.pvalue
        All_Seeds_Stats.append(stats)
        All_Seeds_Stats.append(pvals)
    return [All_Seeds_Stats, All_Seeds_Pvals]

In [ ]:
nov_exp_results = run_permutation_test(nov_data, exp_data)
nov_int_results = run_permutation_test(nov_data, int_data)

with open("../results/nov_exp_permutation_results.pkl", "wb") as f:
    pickle.dump(nov_exp_results, f)

with open("../results/nov_int_permutation_results.pkl", "wb") as f:
    pickle.dump(nov_int_results, f)

seed 0 permutation testing...


Processing voxels:  23%|██████▋                      | 52670/228483 [3:36:07<11:40:07,  4.19it/s]